### 馬路聲音與非車禍聲音結合測試

In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
import time as t
import wave
import math
from numpy.fft import fft
import sys
import re
import scipy.io as scio
import tensorflow_io as tfio
import librosa
from tensorflow.python.ops.numpy_ops import np_config

In [ ]:
filepath1 = 'car_crash_dataset/soundsnap/bird/4081-barn_swallow_song_cut4_teesdale_140606.wav'
filepath2 = 'car_crash_dataset/ESC50/馬路聲音.wav'

import numpy as np
import wave

def read_wave(file):
    with wave.open(file, 'rb') as f:
        params = f.getparams()
        nchannels, sampwidth, framerate, nframes = params[:4]
        str_data = f.readframes(nframes)
        wave_data = np.frombuffer(str_data, dtype=np.int16)
        wave_data = wave_data.reshape(-1, nchannels)
    return wave_data, framerate

def write_wave(file, data, framerate):
    with wave.open(file, 'wb') as f:
        nchannels = data.shape[1] if len(data.shape) > 1 else 1
        sampwidth = 2  # 16-bit audio
        nframes = data.shape[0]
        comptype = "NONE"
        compname = "not compressed"
        f.setparams((nchannels, sampwidth, framerate, nframes, comptype, compname))
        f.writeframes(data.tobytes())

def mix_waves(wave1, wave2):
    # Ensure both waves have the same number of channels and frames
    min_length = min(len(wave1), len(wave2))
    wave1 = wave1[:min_length]
    wave2 = wave2[:min_length]
    
    # Mix waves by adding the samples
    mixed_wave = wave1 + wave2

    # Normalize to prevent clipping
    max_val = np.max(np.abs(mixed_wave))
    if max_val > 32767:
        mixed_wave = mixed_wave * (32767.0 / max_val)
    
    return mixed_wave.astype(np.int16)

# 讀取兩個聲音檔
wave1, framerate1 = read_wave(filepath1)
wave2, framerate2 = read_wave(filepath2)

# 確保兩個聲音檔有相同的採樣率
if framerate1 != framerate2:
    print(framerate1, framerate2)
    if framerate1 != framerate2:
        #raise ValueError("Sample rates do not match")
        target_rate = min(framerate1, framerate2)  # 選擇較低的採樣率
        if framerate1 != target_rate:
            wave1 = wave1.astype(np.float32)
            wave1 = librosa.resample(wave1, orig_sr=framerate1, target_sr=target_rate)
            wave1 = wave1.astype(np.int16)
        if framerate2 != target_rate:
            wave2 = wave2.astype(np.float32)
            wave2 = librosa.resample(wave2, orig_sr=framerate2, target_sr=target_rate)
            wave2 = wave2.astype(np.int16)
        framerate1 = framerate2 = target_rate  # 更新採樣率

# 混和音頻
mixed_wave = mix_waves(wave1, wave2)
print(mixed_wave.size)
abs_wave_data = [abs(i)for i in mixed_wave]
print(abs_wave_data[0])
wave_data = mixed_wave*1.0/(max(abs_wave_data))

# 寫入混合後的音頻文件
write_wave('new_wave.wav', mixed_wave, framerate1)


#### 聲音預處理端測試資料產生以及中間值輸出 (直接執行下方所有block即可)

In [3]:
def findsize(path1, delay, index):
    wlen=1024
    inc=128

    f = wave.open(f'{path1}', "rb")
    params = f.getparams()
    nchannels, sampwidth, framerate, nframes = params[:4]
    str_data = f.readframes(nframes)
    wave_data = np.frombuffer(str_data, dtype=np.short)
    wave_data = wave_data*1.0/(max(abs(wave_data)))
    time = np.arange(0, wlen) * (1.0 / framerate)
    fixed_signal = np.zeros(len(wave_data))
    # if delay != 0:
    #     zeros=np.zeros(((wlen//11*delay),)) #在訊號前加0增加訓練集
    #     fixed_signal=np.concatenate((zeros,wave_data)) #填補後的信號記爲fixed_signal
    # else:
    fixed_signal = wave_data
    #print(wave_data.dtype)
    signal_length=len(fixed_signal) #信號總長度
    if signal_length<=wlen: #若信號長度小於一個幀的長度，則幀數定義爲1
        nf=1
    else: #否則，計算幀的總長度
        nf=int(np.ceil((1.0*signal_length-wlen+inc)/inc))
    pad_length=int((nf-1)*inc+wlen) #所有幀加起來總的鋪平後的長度
    zeros=np.zeros((pad_length-signal_length,)) #不夠的長度使用0填補，類似於FFT中的擴充數組操作
    pad_signal=np.concatenate((fixed_signal,zeros)) #填補後的信號記爲pad_signal
    indices=np.tile(np.arange(0,wlen),(nf,1))+np.tile(np.arange(0,nf*inc,inc),(wlen,1)).T  #相當於對所有幀的時間點進行抽取，得到nf*nw長度的矩陣
    indices=np.array(indices,dtype=np.int32) #將indices轉化爲矩陣
    #print(indices.shape)
    size = indices.shape[0]
    return size

In [4]:
# 十進制轉回二進制
def float_to_fixed_binary(value, integer_bits = 5, decimal_bits = 10):

    #略小於數值則為0
    if value < 9.7656*(10**-4) and value > -9.7656*(10**-4):
        binary_representation = '0000000000000000'
        return binary_representation

    # 考慮正負號
    sign = 0 if value >= 0 else 1
    abs_value = abs(value)

    # 將浮點數轉換為固定點表示法
    fixed_integer = int(abs_value * (2 ** decimal_bits))
    # 限制整數部分的位數
    #fixed_integer = fixed_integer % (2 ** (integer_bits + decimal_bits))

    # 考慮正負號
    sign_bit = '0' if sign == 0 else '1'
    
    # 生成二進位表示法
    binary_representation = format(fixed_integer, f'0{integer_bits + decimal_bits}b')
    
    # 如果是負數，進行二補數表示法的轉換
    if sign == 1:
        binary_representation = ''.join('1' if bit == '0' else '0' for bit in binary_representation)

        # 進行二進位加法，將結果加 1
        carry = 1
        for i in range(len(binary_representation) - 1, -1, -1):
            bit_sum = int(binary_representation[i]) + carry
            binary_representation = binary_representation[:i] + str(bit_sum % 2) + binary_representation[i+1:]
            carry = bit_sum // 2

    # 加上正負號位元
    binary_representation = f'{sign_bit}{binary_representation}'

    return binary_representation

In [5]:
def verilog_testbench(soundsignal):
    for i in range(soundsignal.shape[0]):
        if soundsignal[i] < -31:
            print(i, soundsignal[i])
            raise ValueError("data too small!")
        elif soundsignal[i] > 31:
            raise ValueError("data too big!")
        else:
            binarytestbench = float_to_fixed_binary(soundsignal[i])
        print(f'        #30 sound_input = 16\'b{binarytestbench}; sound_input_Rready = 1\'b1;')
        print('        #30 sound_input_Rready = 1\'b0;')
    return

In [6]:
def add_white_noise(signal, noise_factor = 0.005):
    noise = np.random.normal(loc = 0, scale = signal.std(), size = len(signal))
    augmented_signal = signal + noise*noise_factor
    return augmented_signal

def audioProcessing(path1, delay, designated = 0, maxindex = 167, addenvnoise = 0):
    wlen=1024
    inc=128

    f = wave.open(f'{path1}', "rb")
    params = f.getparams()
    nchannels, sampwidth, framerate, nframes = params[:4]
    str_data = f.readframes(nframes)
    wave_data = np.frombuffer(str_data, dtype=np.short)
    wave_data = wave_data*1.0/(max(abs(wave_data)))
    if delay == 1: #add white noise

        wave_data = add_white_noise(wave_data, 0.5)

    elif delay == 3:
        wave_data = add_white_noise(wave_data, 0.3)
    elif delay == 2:
        wave_data = add_white_noise(wave_data, 0.4)
    time = np.arange(0, wlen) * (1.0 / framerate)
    fixed_signal = np.zeros(len(wave_data))
    fixed_signal = wave_data
    #print(wave_data.dtype)
    signal_length=len(fixed_signal) #信號總長度
    if signal_length<=wlen: #若信號長度小於一個幀的長度，則幀數定義爲1
        nf=1
    else: #否則，計算幀的總長度
        nf=int(np.ceil((1.0*signal_length-wlen+inc)/inc))
    pad_length=int((nf-1)*inc+wlen) #所有幀加起來總的鋪平後的長度
    zeros=np.zeros((pad_length-signal_length,)) #不夠的長度使用0填補，類似於FFT中的擴充數組操作
    pad_signal=np.concatenate((fixed_signal,zeros)) #填補後的信號記爲pad_signal
    indices=np.tile(np.arange(0,wlen),(nf,1))+np.tile(np.arange(0,nf*inc,inc),(wlen,1)).T  #相當於對所有幀的時間點進行抽取，得到nf*nw長度的矩陣
    #print(indices[:2])
    indices=np.array(indices,dtype=np.int32) #將indices轉化爲矩陣
    frames=pad_signal[indices] #得到幀信號
    totaltime = np.linspace(0, 1.0/framerate*nframes, signal_length)


    maxamplitude = maxindex #max amplitude 所在的窗
    plt_time = time + inc /framerate*maxamplitude

    np.set_printoptions(threshold=sys.maxsize)
    print(frames.shape[1])
    #print(type(fixed_signal))
    #print(fixed_signal)
    print('original: ', frames[designated])
    verilog_testbench(frames[designated])
    
    b = np.zeros((frames.shape[0], frames.shape[1]))
    for i in range(0, frames.shape[0]):
        windown=np.hamming(wlen)  #調用漢明窗
        a=frames[i:i+1]
        b[i]=a[0]*windown
    
    print('window: ', b[designated])

    #FFT

    Xtest = fft(b[0])
    X = np.zeros((b.shape[0], len(Xtest)),dtype=np.complex_)

    for i in range(0, b.shape[0]):
        X[i] = fft(b[i])
        N = len(X[0])
        n = np.arange(N)
        T = N/framerate
        freq = n/T 

    n_oneside = N//2


    # get the one side frequency
    f_oneside = freq[:n_oneside]

    # normalize the amplitude
    X_oneside = np.zeros((X.shape[0], f_oneside.shape[0]),dtype=np.complex_)
    for i in range (0, X.shape[0]):
        X_oneside[i] =X[i][:n_oneside]/n_oneside

    
    square_X = np.zeros((X_oneside.shape[0], X_oneside.shape[1]))
    square_X = np.square(np.abs(X_oneside))

    print('fft: ', square_X[designated])
    
    #頻率點數量
    fp = n_oneside
    #設計濾波器的最低頻率
    fl = freq[0]
    #設計濾波器的最高頻率
    fh = freq[fp]
    #print(f'maximum freq: {fh}')
    #最低頻率對應的mel頻率
    melfl = 2595.0 * np.log10(1 + fl/700.0)
    #最高頻率對應的mel頻率
    melfh = 2595.0 * np.log10(1 + fh/700.0)
    # melfl 到 melfh 之間的濾波器個數
    p = 64
    #間隔點頻率(包括最低頻點及最高頻點)
    MelF = np.linspace(melfl, melfh, p+2)
    #將mel頻率轉回實際頻率
    F = 700.0 * (10 ** (MelF/2595.0) - 1)
    bank = np.zeros((p, fp))


    for m in range(1, p+1):
        F_left = F[m - 1]
        F_mid = F[m]
        F_right = F[m + 1]
        for k in range(0, fp):
            
            if f_oneside[k] >= F_left and f_oneside[k] <= F_mid:
                bank[m - 1][k] = (f_oneside[k] - F_left)/(F_mid - F_left)
            elif f_oneside[k] > F_mid and f_oneside[k] <= F_right:
                bank[m - 1][k] = (F_right - f_oneside[k])/(F_right - F_mid)        


    mel_X = np.matmul(square_X, np.transpose(bank))

    print('mel: ', mel_X[designated])

    #min_positive_float = np.finfo(float).eps
    min_positive_float = 10**-7
    for i in range(mel_X.shape[0]):
        for j in range(mel_X.shape[1]):
            if mel_X[i][j] < 10**-7:
                mel_X[i][j] = min_positive_float

    
    log_X = np.log10(mel_X)
    for i in range(log_X.shape[0]):
        for j in range(log_X.shape[1]):
            if log_X[i][j] < -7:
                print("minimum overflow")
            if  log_X[i][j] > 7:
                print("maximum overflow")
    
    print('log10: ', log_X[designated])
    
    return log_X

In [7]:
signal1 = np.zeros((1, 64, 64)) #car_horn 8音檔
index = 1
maxindex = 0
#原始音檔 & 修改音檔增加訓練資料
path1 = 'car_crash_dataset/soundsnap/0s/11874-CAR_WINDOW_SMASH_1.wav'
for i in range(1): #i 表示平移量
    signal_size = findsize(path1, i, index)
    temp_signal = np.zeros((signal_size, 64))
    temp_signal = audioProcessing(path1, i, index, maxindex)
    num_matrices = temp_signal.shape[0] // 64
    # 使用 numpy.reshape 將原始陣列轉換為三維矩陣
    signal1 = np.append(signal1, temp_signal[:num_matrices * 64, :].reshape((num_matrices, 64, 64)), axis = 0)
    index += 1

1024
original:  [-6.10370190e-05 -7.29087191e-02 -1.32847072e-01 -7.01925718e-04
 -6.14947966e-02  9.45310831e-01  1.22074038e-04 -2.57271035e-02
  2.73415326e-01  2.44148076e-04  1.64403211e-01 -4.68764306e-01
 -3.05185095e-05 -1.95135350e-01  2.57789850e-01 -2.74666585e-04
  2.09387494e-01 -8.12524796e-01  8.23999756e-04  8.47193823e-02
  1.71880245e-01  6.10370190e-04  2.61787774e-01  6.71895505e-01
  6.71407208e-04 -3.77208777e-02 -2.34687338e-02 -5.79851680e-04
 -4.01013215e-02  9.84374523e-01 -6.10370190e-05 -5.92669454e-02
 -9.60997345e-01 -3.35703604e-04 -1.60344249e-01  6.24713889e-02
 -5.49333171e-04 -1.14017151e-01 -8.90682699e-01 -7.93481246e-04
 -1.98919645e-01  4.68459120e-02 -5.79851680e-04 -9.07925657e-02
  1.32786035e-01 -1.52592547e-04 -7.38547929e-02 -9.37559130e-01
 -2.74666585e-04 -5.90838343e-02  6.56239509e-01 -4.27259133e-04
 -8.36207160e-02  1.24973296e-01 -2.44148076e-04 -1.16275521e-02
  5.70299387e-01  3.05185095e-05 -7.96838282e-02  2.81228065e-01
 -3.05185